# Create and manage a batch endpoint for inferencing

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../jobs/configuration.ipynb) 

- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create a batch endpoint from Python SDK
- Create deployments on that endpoint from Python SDK
- Test a deployment with a sample request

**Motivations** - This notebook explains how to create an online endpoint and manage deployments on that endpoint. An endpoint is an HTTPS endpoint that clients can call to receive the inferencing (scoring) output of a trained model. Online endpoints are endpoints that are used for online (real-time) inferencing. 

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    Model,
    Environment,
    BatchRetrySettings,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2. Create Batch Endpoint
Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `identity`- The managed identity configuration for accessing Azure resources for endpoint provisioning and inference.
- `defaults` - Default settings for the endpoint.
   - `deployment_name` - Name of the deployment that will serve as the default deployment for the endpoint.
- `description`- Description of the endpoint.

## 2.1 Configure the endpoint

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = "mnist-batch-" + endpoint_suffix

Let's configure the endpoint:

In [ ]:
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for scoring images from the MNIST dataset.",
    tags={"type": "deep-learning"},
)

## 2.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 3. Registering the model

### 3.1 About the model

We are going to deploy a model created using Torch to solve the typical MNIST classification problem.

### 3.2 Registering the model in the workspace

We need to register the model in order to use it with Azure Machine Learning:

In [ ]:
model_name = "mnist-classification-torch"
model_local_path = "./mnist/model/"

In [ ]:
if not any(filter(lambda m: m.name == model_name, ml_client.models.list())):
    print(f"Model {model_name} is not registered. Creating...")
    model = ml_client.models.create_or_update(
        Model(name=model_name, path=model_local_path, type=AssetTypes.CUSTOM_MODEL)
    )

Let's get a reference to the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

# 3. Create a deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing.

### 4.1 Creating an scoring script to work with the model

In [ ]:
%%writefile mnist/code/batch_driver.py

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from azureml.core import Model


def init():
    global g_tf_sess

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder (./azureml-models)
    # Please provide your model's folder name if there's one
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # contruct graph to execute
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(os.path.join(model_path, "mnist-tf.model.meta"))
    g_tf_sess = tf.Session(config=tf.ConfigProto(device_count={"GPU": 0}))
    saver.restore(g_tf_sess, os.path.join(model_path, "mnist-tf.model"))


def run(mini_batch):
    print(f"run method start: {__file__}, run({mini_batch})")
    resultList = []
    in_tensor = g_tf_sess.graph.get_tensor_by_name("network/X:0")
    output = g_tf_sess.graph.get_tensor_by_name("network/output/MatMul:0")

    for image in mini_batch:
        # prepare each image
        data = Image.open(image)
        np_im = np.array(data).reshape((1, 784))
        # perform inference
        inference_result = output.eval(feed_dict={in_tensor: np_im}, session=g_tf_sess)
        # find best probability, and add to result list
        best_result = np.argmax(inference_result)
        resultList.append([os.path.basename(image), best_result])

    return pd.DataFrame(resultList)

### 4.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [ ]:
compute_name = "cpu-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    print(f"Compute {compute_name} is not created. Creating...")
    compute_cluster = AmlCompute(
        name=compute_name,
        description="CPU cluster compute",
        min_instances=0,
        max_instances=2,
    )
    ml_client.compute.begin_create_or_update(compute_cluster).result()

### 4.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

In [ ]:
env = Environment(
    conda_file="./mnist/environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)

## 3.1 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_path`- Path to the source code directory for scoring the model
- `scoring_script` - Relative path to the scoring file in the source code directory
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.
- `logging_level`- The log verbosity level.	Allowed values are `warning`, `info`, `debug`. Default is `info`.

In [ ]:
deployment = BatchDeployment(
    name="mnist-torch-dpl",
    description="A deployment using Torch to solve the MNIST classification dataset.",
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="./mnist/code/", scoring_script="batch_driver.py"
    ),
    environment=env,
    compute=compute_name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

## 3.2 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.begin_create_or_update(deployment).result()

Let's update the default deployment name in the endpoint:

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

We can see the endpoint URL as follows:

In [ ]:
print(f"The default deployment is {endpoint.defaults.deployment_name}")

## 3.3 Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present
- `deployment_name` - Name of the specific deployment to test in an endpoint

### 3.3.1 Invoke the endpoint

Let's use public data available in an Azure Storage Account.

In [ ]:
input = Input(
    type="uri_folder",
    path="https://pipelinedata.blob.core.windows.net/sampledata/mnist",
)

Let's now invoke the endpoint for batch scoring job:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=input,
)

### 3.3.2 Get the details of the invoked job
Let us get details and logs of the invoked job

In [ ]:
ml_client.jobs.get(job.name)

We can wait for the job to finish using the following code:

In [ ]:
from time import sleep

print(f"Waiting for batch deployment job {job.name}", end="")
while ml_client.jobs.get(name=job.name).status not in ["Completed", "Failed"]:
    sleep(10)
    print(".", end="")

print(" [DONE]")

#### 3.3.3 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

The outputs generated by the deployment job will be placed in an output named `score`:

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

In [ ]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv",
    header=None,
    names=["file", "class"],
    sep=" ",
)
score

# 4. Clean up Resources
Delete endpoint

In [ ]:
ml_client.batch_endpoints.begin_delete(name=endpoint_name)